In [40]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)

In [41]:

import nltk
import gensim
        
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

def reviews():            
    r_i = 0
    review_item_iterator = iter(parse("../download/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
    for it in range(1000): #range(1000000):
        if it%10000==0:
            print("it="+str(it))
        review_item = next(review_item_iterator)
        r_i = r_i + 1
        if r_i%10000==0:
            print(str(r_i))
            print(preprocess_sentence)
        if "Cell Phones" in review_item["categories"]:
            reviewText = review_item["reviewText"]
            s_i = 0  
            for sentence in sent_detector.tokenize(reviewText):
                s_i = s_i + 1

                #label = review_item["asin"] + "_" + review_item["reviewerID"] + "_" + str(s_i)
                preprocess_sentence = gensim.utils.simple_preprocess(sentence)

                yield preprocess_sentence
                

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def nextTokenizedSentence():
    yield [token for token in tokens if token not in stop and token not in exclude]



In [42]:
def nextTaggedSentence(cursor, textFieldName):
    for tokenizedSentence in reviews():
        #print(tokenizedSentence)
        tagged = nltk.pos_tag(tokenizedSentence)
        yield tagged

In [43]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases



In [44]:
def nextNounPhrase(cursor, textFieldName):
    for taggedSentence in nextTaggedSentence(cursor, textFieldName):
        grammar = r"""
          NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
            """
        cp = nltk.RegexpParser(grammar)
        if taggedSentence:
            noun_phrase_tagged = cp.parse(taggedSentence)
            noun_phrases=[]
            nouns=traverse(noun_phrase_tagged,noun_phrases)
            #print(nouns)
            for noun in nouns:
                yield noun
 

import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    doc_complete = [review_doc["buyerTranslationFeedback"] 
                    for review_doc in my_collection.find() 
                    if "buyerTranslationFeedback" in review_doc.keys() ]
else:
    my_collection = mydb['amazon_product_reviews']
    doc_complete = [review_doc["reviewText"] 
                    for review_doc in my_collection.find({ "categories" : "Cell Phones" }) ]
print("count of reviews: "+ str(len(doc_complete)))



import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    cursor = my_collection.find() 
    textFieldName = "buyerTranslationFeedback"
else:
    my_collection = mydb['amazon_product_reviews']
    cursor = my_collection.find({ "categories" : "Cell Phones" })
    textFieldName = "reviewText"


In [49]:
from nltk import FreqDist

fdist = FreqDist(nextNounPhrase(cursor, textFieldName)) 
frequent_words_freq = fdist.most_common(100)
print(frequent_words_freq )
#fdist.plot()
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)
import numpy as np
np_frequent_words = np.asarray([w for (w,c) in frequent_words_freq])
print(np_frequent_words)

it=0
[('phone', 15), ('product', 6), ('consumer', 3), ('phones', 3), ('reviews', 3), ('galaxy', 2), ('settings', 2), ('call', 2), ('samsung', 2), ('problems', 2), ('price', 2), ('line', 2), ('seller', 2), ('star', 1), ('plan', 1), ('computer', 1), ('nerd', 1), ('lot', 1), ('fun', 1), ('cellular', 1), ('network', 1), ('anyone', 1), ('wanting', 1), ('needs', 1), ('lots', 1), ('patience', 1), ('smarts', 1), ('access', 1), ('entries', 1), ('apn', 1), ('file', 1), ('years', 1), ('service', 1), ('wife', 1), ('att', 1), ('store', 1), ('card', 1), ('minutes', 1), ('tile', 1), ('floor', 1), ('displays', 1), ('operating', 1), ('system', 1), ('sellers', 1), ('breed', 1), ('number', 1), ('amazon', 1), ('clone', 1), ('buyers', 1), ('names', 1), ('sale', 1), ('ad', 1), ('galazy', 1), ('photo', 1), ('quality', 1), ('ve', 1), ('ui', 1), ('dial', 1), ('pad', 1), ('wait', 1), ('seconds', 1), ('piece', 1), ('crap', 1), ('rom', 1), ('kernel', 1), ('anything', 1), ('kitkat', 1), ('software', 1), ('buy', 1)

In [ ]:
import nltk
from nltk.cluster.kmeans import KMeansClusterer
nc = 10
data = np_array

kclusterer = KMeansClusterer(nc, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(data, assign_clusters=True)

In [46]:
features=  [ {"feature" : key} for key in frequent_words]
print(features)

[{'feature': 'phone'}, {'feature': 'product'}, {'feature': 'consumer'}, {'feature': 'phones'}, {'feature': 'reviews'}, {'feature': 'galaxy'}, {'feature': 'settings'}, {'feature': 'call'}, {'feature': 'samsung'}, {'feature': 'problems'}, {'feature': 'price'}, {'feature': 'line'}, {'feature': 'seller'}, {'feature': 'star'}, {'feature': 'plan'}, {'feature': 'computer'}, {'feature': 'nerd'}, {'feature': 'lot'}, {'feature': 'fun'}, {'feature': 'cellular'}, {'feature': 'network'}, {'feature': 'anyone'}, {'feature': 'wanting'}, {'feature': 'needs'}, {'feature': 'lots'}, {'feature': 'patience'}, {'feature': 'smarts'}, {'feature': 'access'}, {'feature': 'entries'}, {'feature': 'apn'}, {'feature': 'file'}, {'feature': 'years'}, {'feature': 'service'}, {'feature': 'wife'}, {'feature': 'att'}, {'feature': 'store'}, {'feature': 'card'}, {'feature': 'minutes'}, {'feature': 'tile'}, {'feature': 'floor'}, {'feature': 'displays'}, {'feature': 'operating'}, {'feature': 'system'}, {'feature': 'sellers'},

In [47]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['features']
try:
    mydb.features.drop()
finally:
    result = mydb.features.create_index([('feature', pymongo.ASCENDING)],unique=True)

In [48]:
my_collection.insert_many(features)